In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../input/dennisdutchlangdataset/dataset.csv")

df["title"] = df.text.apply(lambda x: x.split("\n")[0].strip())
df["text"] = df.text.apply(lambda x: "\n".join(x.split("\n")[1:]).strip())

In [3]:
!pip install git+https://github.com/LIAAD/yake -q

In [4]:
import yake
from tqdm.notebook import tqdm

all_keywords = []
for text in tqdm(df.text.tolist()):

    kw_extractor = yake.KeywordExtractor(lan="nl")
    extracted = kw_extractor.extract_keywords(text)

    keywords = []

    for kw in extracted:
        keywords.append(kw)
        
    all_keywords.append(keywords)
    
df["keywords"] = all_keywords

  0%|          | 0/300 [00:00<?, ?it/s]

In [5]:
model_name = "yhavinga/gpt2-medium-dutch"

keyword_number = 3

In [6]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/207 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

In [7]:
df.columns

Index(['Unnamed: 0', 'text', 'filenames', 'title', 'keywords'], dtype='object')

In [8]:
import re

training_texts = []

for record in df.to_dict("record"):
    
    keywords = [i[0].strip(",") for i in record["keywords"][:keyword_number]]
    text = record["text"]
    text = re.sub("[\n]+", "\n\n", text)
    title = record["title"]
    
    training_text = title + "\n" + ", ".join(keywords) + "\n\n#######\n\n" + text
    training_text = tokenizer.decode(tokenizer.encode(training_text)[:1024])
    
    training_texts.append(training_text)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """


In [9]:
df["training_text"] = training_texts

train = df.iloc[:295]
test = df.iloc[295:]

In [10]:
text = "\n\n\n---\n\n\n".join(training_texts)

f = open("train.txt", "w", encoding="utf-8")
f.write(text)
f.close()

In [11]:
!pip install simpletransformers -q

In [12]:
import logging

from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



In [13]:
from simpletransformers.language_modeling import LanguageModelingModel


train_args = {
    "reprocess_input_data": True,
    "num_train_epochs": 2,
    "save_eval_checkpoints": False,
    "block_size": 200,
    "max_seq_length": 200,
    # "save_model_every_epoch": False,
    "mlm": False,
    "learning_rate": 1e-4,
    "train_batch_size": 1,
    "gradient_accumulation_steps":2,
    "dataset_type": "simple",
    "logging_steps": 100,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 3000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": True,
    "sliding_window": True,
    "use_multiprocessing": False,
    "evaluate_during_training": False,
    "num_beams": 2
}

In [14]:
#TODO: Find a way to resize tokenizer instead of using standard one
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


class SimpleDataset(Dataset):
    def __init__(
        self,
        tokenizer,
        file_path,
    ):

        with open(file_path, encoding="utf-8") as f:
            lines = [
                tokenizer.encode(
                    ele,
                )
                for ele in f.read().split("\n\n\n---\n\n\n")
            ]

        self.examples = lines


    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


    
def load_and_cache_examples(self, file_path, verbose=None, evaluate=None, silent=None):
    return SimpleDataset(self.tokenizer, file_path)


LanguageModelingModel.load_and_cache_examples = load_and_cache_examples

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [15]:
model = LanguageModelingModel("gpt2", model_name, args=train_args)

Downloading:   0%|          | 0.00/207 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/487k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

In [16]:
import torch

model.train_model(
    "train.txt", 
)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/300 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/simpletransformers/language_modeling/language_modeling_model.py:864: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


Running Epoch 1 of 2:   0%|          | 0/300 [00:00<?, ?it/s]

(300, 2.2321565985679626)

In [17]:
import logging
from simpletransformers.language_generation import LanguageGenerationModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = LanguageGenerationModel("gpt2", "outputs", args={"max_length": 512, "top_k":50, "num_return_sequences":1})

In [18]:
written_texts = test.training_text.apply(lambda x: x.split("#######")[0] + "#######").tolist()
texts = []

for written_text in written_texts:
    texts.append(model.generate(written_texts[0], verbose=False)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [19]:
written_texts

['Welke lamp kies je voor welke kamer?\nlicht, lamp, neus en lippen\n\n#######',
 'Tips voor het promoten van je onderneming\nwebshop, zorgt niet perse, onderneming zorgt\n\n#######',
 'Nieuwe keuken? Hier moet je op letten\nkeuken, plan om binnenkort, nieuwe keuken\n\n#######',
 'Heb jij een favoriet woonmerk?\npersoon heeft duidelijk, duidelijk een favoriet, merk\n\n#######',
 'Nieuw internetabonnement afsluiten? Hier moet je zeker op letten!\nnieuw internetabonnement, verbinding, provider\n\n#######']

In [20]:
output_text = ""
for text in texts:
    
    splitted = text.split("#######")
    output_text += f"Input: {splitted[0]}\n----\noutput: {splitted[1]} \n\n\n ****************** \n\n\n"

In [21]:
file = open("examples_01.txt", "w", encoding="utf-8")
file.write(output_text)
file.close()

In [22]:
model

In [23]:
model.model.save_pretrained("model_1.h5")